In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

# Data dictionary for final cleaned data

Data from 2016 for number of uninsured/insured within a specific intersection of race/income/age/sex categories for all US states.

The raw data was pulled from the US Census' [2008-2018 Small Area Health Insurance Estimates](https://www.census.gov/data/datasets/time-series/demo/sahie/estimates-acs.html) via the American Community Survey and is stored at path "../raw_data/sahie_2016.csv".

The cleaned data preserving counties in each state is stored at the path "../data/2016_us_counties_insurance.csv".

The cleaned data not preserving counties in each state is stored at the path "../data/2016_us_insurance.csv".

### preserved counties

| column name | meaning |
| ----------- | ------- |
| year | Year the data estimates are for |
| statefips | Unique 2-digit FIPS code for each state |
| countyfips | Unique 3-digit FIPS code for each county within a state |
| geocat | Geography category:<br>40 - State geographic identifier<br>50 - County geographic identifier) |
| agecat | Age category:<br>0 - Under 65 years<br>1 - 18 to 64 years<br>2 - 40 to 64 years<br>3 - 50 to 64 years<br>4 - Under 19 years<br>5 - 21 to 64 years |
| racecat | Race category:<br>0 - All races<br>1 - White alone, not Hispanic<br>2 - Black alone, not Hispanic<br>3 - Hispanic (any race) |
| sexcat | Sex category:<br>0 - Both sexes<br>1 - Male<br>2 - Female |
| iprcat | Income category:<br>0 - All income levels<br>1 - At or below 200% of poverty<br>2 - At or below 250% of poverty<br>3 - At or below 138% of poverty<br>4 - At or below 400% of poverty<br>5 - Between 138% - 400%  of poverty |
| NIPR | Number in demographic group for income category |
| nipr_moe | Margin of error (MOE) for NIPR |
| NUI | Number uninsured  | 
| nui_moe | MOE for NUI |
| NIC | Number insured |
| nic_moe | MOE  for NIC |
| PCTUI | Percent uninsured in demographic group for income category |
| pctui_moe | MOE  for PCTUI |
| PCTIC |  Percent insured in demographic group for income category |
| pctic_moe | MOE  for PCTIC |
| PCTELIG | Percent uninsured in demographic group for all income levels |
| pctelig_moe | MOE  for PCTELIG |
| PCTLIIC | Percent insured in demographic group for all income levels | 
| pctliic_moe | MOE  for PCTLIIC |
| state_name | State name |
| county_name | County name |
| GEOID20 | Unique 5-digit county identifier (statefips + countyfips) |

### did not preserve counties

| column name | meaning |
| ----------- | ------- |
| year | Year the data estimates are for |
| state_name | State name |
| STATEFP20 | Unique 2-digit FIPS code for each state |
| agecat | Age category:<br>Under 65 years<br>18 to 64 years<br>40 to 64 years<br>50 to 64 years<br>Under 19 years<br> 21 to 64 years |
| racecat | Race category:<br>All - All races<br>White - White alone, not Hispanic<br>Black - Black alone, not Hispanic<br>Hispanic - Hispanic (any race) |
| sexcat | Sex category:<br>Both - Both sexes<br>Male<br>Female |
| iprcat | Income category:<br>All - All income levels<br><=200% - At or below 200% of poverty<br><=250% - At or below 250% of poverty<br><=138% - At or below 138% of poverty<br><=400 - At or below 400% of poverty<br>138 to 400% - Between 138% - 400%  of poverty |
| NUI | Number uninsured  |
| NIC | Number insured |

# Data wrangling

## Load data

In [3]:
sahie_2016 = pd.read_csv("../raw_data/sahie_2016.csv", header=68, dtype={"statefips": object,
                                                                        "countyfips": object})

/Users/carolyn/opt/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
sahie_2016

,year,version,statefips,countyfips,geocat,agecat,racecat,sexcat,iprcat,NIPR,nipr_moe,NUI,nui_moe,NIC,nic_moe,PCTUI,pctui_moe,PCTIC,pctic_moe,PCTELIG,pctelig_moe,PCTLIIC,pctliic_moe,state_name,county_name,Unnamed: 25
0,2016,,01,000,40,0,0,0,0,3973078,0,427972,12298,3545106,12298,10.8,0.3,89.2,0.3,10.8,0.3,89.2,0.3,Alabama ...,,NaN
1,2016,,01,000,40,0,0,0,1,1514292,13706,274697,9339,1239595,14056,18.1,0.6,81.9,0.6,6.9,0.2,31.2,0.4,Alabama ...,,NaN
2,2016,,01,000,40,0,0,0,2,1876603,13894,319119,10099,1557484,14769,17,0.5,83,0.5,8,0.3,39.2,0.4,Alabama ...,,NaN
3,2016,,01,000,40,0,0,0,3,1048013,12789,201733,7856,846280,12602,19.2,0.7,80.8,0.7,5.1,0.2,21.3,0.3,Alabama ...,,NaN
4,2016,,01,000,40,0,0,0,4,2726753,13798,386721,11378,2340032,15930,14.2,0.4,85.8,0.4,9.7,0.3,58.9,0.4,Alabama ...,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320293,2016,,56,045,50,5,0,2,1,423,45,123,32,300,41,29.1,6.5,70.9,6.5,6.6,1.7,16.1,2.2,Wyoming ...,Weston County,NaN
320294,2016,,56,045,50,5,0,2,2,558,50,154,37,404,47,27.6,5.9,72.4,5.9,8.3,2,21.7,2.5,Wyoming ...,Weston County,NaN
320295,2016,,56,045,50,5,0,2,3,274,39,85,25,189,33,31,7.5,69,7.5,4.6,1.3,10.2,1.8,Wyoming ...,Weston County,NaN
320296,2016,,56,045,50,5,0,2,4,951,60,217,47,734,61,22.8,4.6,77.2,4.6,11.7,2.5,39.5,3.3,Wyoming ...,Weston County,NaN


## Wrangle version of US data preserving counties

In [5]:
def drop_unnec(df, ver=False):
    if ver == True:
        df.drop(["version", "Unnamed: 25"], axis=1, inplace=True)
    else:
        df.drop(["Unnamed: 25"], axis=1, inplace=True)
    return df

In [6]:
# check if anything is stored in version
sahie_2016["version"].unique()

array(['        '], dtype=object)

In [7]:
sahie_2016 = drop_unnec(sahie_2016, True)

In [8]:
sahie_2016.dtypes

year            int64
statefips      object
countyfips     object
geocat          int64
agecat          int64
racecat         int64
sexcat          int64
iprcat          int64
NIPR           object
nipr_moe       object
NUI            object
nui_moe        object
NIC            object
nic_moe        object
PCTUI          object
pctui_moe      object
PCTIC          object
pctic_moe      object
PCTELIG        object
pctelig_moe    object
PCTLIIC        object
pctliic_moe    object
state_name     object
county_name    object
dtype: object

In [9]:
# convert columns to appropriate data types
cols = ["NIPR", "nipr_moe", "NUI", "nui_moe", "NIC", "nic_moe", "PCTUI", "pctui_moe", "PCTIC", "pctic_moe", "PCTELIG", "pctelig_moe", "PCTLIIC", "pctliic_moe"]

sahie_2016[cols] = sahie_2016[cols].apply(pd.to_numeric, errors="coerce")

In [10]:
sahie_2016.dtypes

year             int64
statefips       object
countyfips      object
geocat           int64
agecat           int64
racecat          int64
sexcat           int64
iprcat           int64
NIPR           float64
nipr_moe       float64
NUI            float64
nui_moe        float64
NIC            float64
nic_moe        float64
PCTUI          float64
pctui_moe      float64
PCTIC          float64
pctic_moe      float64
PCTELIG        float64
pctelig_moe    float64
PCTLIIC        float64
pctliic_moe    float64
state_name      object
county_name     object
dtype: object

In [11]:
# add GEOID20
sahie_2016["GEOID20"] = sahie_2016["statefips"] + sahie_2016["countyfips"]

In [12]:
sahie_2016

,year,statefips,countyfips,geocat,agecat,racecat,sexcat,iprcat,NIPR,nipr_moe,NUI,nui_moe,NIC,nic_moe,PCTUI,pctui_moe,PCTIC,pctic_moe,PCTELIG,pctelig_moe,PCTLIIC,pctliic_moe,state_name,county_name,GEOID20
0,2016,01,000,40,0,0,0,0,3973078.0,0.0,427972.0,12298.0,3545106.0,12298.0,10.8,0.3,89.2,0.3,10.8,0.3,89.2,0.3,Alabama ...,,01000
1,2016,01,000,40,0,0,0,1,1514292.0,13706.0,274697.0,9339.0,1239595.0,14056.0,18.1,0.6,81.9,0.6,6.9,0.2,31.2,0.4,Alabama ...,,01000
2,2016,01,000,40,0,0,0,2,1876603.0,13894.0,319119.0,10099.0,1557484.0,14769.0,17.0,0.5,83.0,0.5,8.0,0.3,39.2,0.4,Alabama ...,,01000
3,2016,01,000,40,0,0,0,3,1048013.0,12789.0,201733.0,7856.0,846280.0,12602.0,19.2,0.7,80.8,0.7,5.1,0.2,21.3,0.3,Alabama ...,,01000
4,2016,01,000,40,0,0,0,4,2726753.0,13798.0,386721.0,11378.0,2340032.0,15930.0,14.2,0.4,85.8,0.4,9.7,0.3,58.9,0.4,Alabama ...,,01000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320293,2016,56,045,50,5,0,2,1,423.0,45.0,123.0,32.0,300.0,41.0,29.1,6.5,70.9,6.5,6.6,1.7,16.1,2.2,Wyoming ...,Weston County,56045
320294,2016,56,045,50,5,0,2,2,558.0,50.0,154.0,37.0,404.0,47.0,27.6,5.9,72.4,5.9,8.3,2.0,21.7,2.5,Wyoming ...,Weston County,56045
320295,2016,56,045,50,5,0,2,3,274.0,39.0,85.0,25.0,189.0,33.0,31.0,7.5,69.0,7.5,4.6,1.3,10.2,1.8,Wyoming ...,Weston County,56045
320296,2016,56,045,50,5,0,2,4,951.0,60.0,217.0,47.0,734.0,61.0,22.8,4.6,77.2,4.6,11.7,2.5,39.5,3.3,Wyoming ...,Weston County,56045


In [13]:
# sahie_2016.to_csv("../data/2016_us_counties_insurance.csv", index=False)

## Wrangle version of US data aggregating counties by state

- Only keeping NIC, NUI data

In [14]:
sahie_2016 = sahie_2016[["year", "state_name", "statefips", "geocat", "agecat", "racecat", "sexcat", "iprcat", "NUI", "NIC"]]

In [15]:
sahie_2016 = sahie_2016.loc[sahie_2016["geocat"] == 40]
sahie_2016.drop(columns=["geocat"], inplace=True)

In [16]:
sahie_2016["state_name"] = sahie_2016["state_name"].str.strip()

In [17]:
sahie_2016.reset_index(inplace=True, drop=True)

In [18]:
sahie_2016.rename(columns={"statefips": "STATEFP20"}, inplace=True)

In [19]:
sahie_2016

,year,state_name,STATEFP20,agecat,racecat,sexcat,iprcat,NUI,NIC
0,2016,Alabama,01,0,0,0,0,427972.0,3545106.0
1,2016,Alabama,01,0,0,0,1,274697.0,1239595.0
2,2016,Alabama,01,0,0,0,2,319119.0,1557484.0
3,2016,Alabama,01,0,0,0,3,201733.0,846280.0
4,2016,Alabama,01,0,0,0,4,386721.0,2340032.0
...,...,...,...,...,...,...,...,...,...
18661,2016,Wyoming,56,5,3,2,1,2670.0,3493.0
18662,2016,Wyoming,56,5,3,2,2,3134.0,4434.0
18663,2016,Wyoming,56,5,3,2,3,1903.0,2309.0
18664,2016,Wyoming,56,5,3,2,4,3906.0,6803.0


In [20]:
# use strings instead of numbers of categories

# agecat
sahie_2016.loc[sahie_2016["agecat"] == 0, "agecat"] = "Under 65 years"
sahie_2016.loc[sahie_2016["agecat"] == 1, "agecat"] = "18 to 64 years"
sahie_2016.loc[sahie_2016["agecat"] == 2, "agecat"] = "40 to 64 years"
sahie_2016.loc[sahie_2016["agecat"] == 3, "agecat"] = "50 to 64 years"
sahie_2016.loc[sahie_2016["agecat"] == 4, "agecat"] = "Under 19 years"
sahie_2016.loc[sahie_2016["agecat"] == 5, "agecat"] = "21 to 64 years"

# racecat
sahie_2016.loc[sahie_2016["racecat"] == 0, "racecat"] = "All"
sahie_2016.loc[sahie_2016["racecat"] == 1, "racecat"] = "White"
sahie_2016.loc[sahie_2016["racecat"] == 2, "racecat"] = "Black"
sahie_2016.loc[sahie_2016["racecat"] == 3, "racecat"] = "Hispanic"

# sexcat
sahie_2016.loc[sahie_2016["sexcat"] == 0, "sexcat"] = "Both"
sahie_2016.loc[sahie_2016["sexcat"] == 1, "sexcat"] = "Male"
sahie_2016.loc[sahie_2016["sexcat"] == 2, "sexcat"] = "Female"

# iprcat
sahie_2016.loc[sahie_2016["iprcat"] == 0, "iprcat"] = "All"
sahie_2016.loc[sahie_2016["iprcat"] == 1, "iprcat"] = "<=200%"
sahie_2016.loc[sahie_2016["iprcat"] == 2, "iprcat"] = "<=250%"
sahie_2016.loc[sahie_2016["iprcat"] == 3, "iprcat"] = "<=138%"
sahie_2016.loc[sahie_2016["iprcat"] == 4, "iprcat"] = "<=400%"
sahie_2016.loc[sahie_2016["iprcat"] == 5, "iprcat"] = "138 to 400%"

In [21]:
sahie_2016

,year,state_name,STATEFP20,agecat,racecat,sexcat,iprcat,NUI,NIC
0,2016,Alabama,01,Under 65 years,All,Both,All,427972.0,3545106.0
1,2016,Alabama,01,Under 65 years,All,Both,<=200%,274697.0,1239595.0
2,2016,Alabama,01,Under 65 years,All,Both,<=250%,319119.0,1557484.0
3,2016,Alabama,01,Under 65 years,All,Both,<=138%,201733.0,846280.0
4,2016,Alabama,01,Under 65 years,All,Both,<=400%,386721.0,2340032.0
...,...,...,...,...,...,...,...,...,...
18661,2016,Wyoming,56,21 to 64 years,Hispanic,Female,<=200%,2670.0,3493.0
18662,2016,Wyoming,56,21 to 64 years,Hispanic,Female,<=250%,3134.0,4434.0
18663,2016,Wyoming,56,21 to 64 years,Hispanic,Female,<=138%,1903.0,2309.0
18664,2016,Wyoming,56,21 to 64 years,Hispanic,Female,<=400%,3906.0,6803.0


In [22]:
# sahie_2016.to_csv("../data/2016_us_insurance.csv", index=False)